In [1]:
import os
from triangulator import create_triangulator_from_files

import sys
sys.path.append('../')
from utils.file_utils import create_capture_from_json

calibration_folder = '../calibration/calibration_data/'
intr_L = os.path.join(calibration_folder, 'calibration_L_frames=623_error=1.5265531081871597.pkl')
intr_R = os.path.join(calibration_folder, 'calibration_R_frames=624_error=0.99089142253755.pkl')
orientation_L = '../calibration/orientation_4.pkl'
orientation_R = '../calibration/orientation_2.pkl'

triangulator = create_triangulator_from_files(
    intr_files=[intr_L, intr_R],
    orientation_files=[orientation_L, orientation_R]
)

Image size is (1920, 1080)
Image size is (1920, 1080)


In [18]:
import numpy as np

zero_point_L = (582, 739)
zero_point_R = (597, 855)

point = (zero_point_L, zero_point_R)

projection = triangulator.projection_matrices[0]
extr = triangulator.extrinsics[0]
cam_matr = triangulator.camera_matrices[0]

# get position with triangulators constructed projection matrix
position_proj = (projection @ [0, 0, 0, 1])
position_proj = position_proj / position_proj[-1]
project_right = triangulator.projection_matrices[1] @ [0, 0, 0, 1]
project_right = project_right / project_right[-1]

# get position with triangulator extrinsics matrix
position_extr = (extr @ [0, 0, 0, 1])
position_extr = cam_matr @ position_extr[:3]
position_extr = position_extr / position_extr[-1]

# create inverse projection matrix
inv_proj = np.linalg.inv(extr)
position_inv = cam_matr @ inv_proj[0:3]
position_inv = position_inv @ [0, 0, 0, 1]
position_inv = position_inv / position_inv[-1]

print('position from projection matrix: ', position_proj)
print('position from right proj matrix: ', project_right)
print('position from extrinsics matrix: ', position_extr)
print('position from inverse matrix   : ', position_inv)

print('triangulated zero point: ', triangulator.triangulate(point))

position from projection matrix:  [572.87374321 743.87228508   1.        ]
position from right proj matrix:  [590.94091106 870.31073854   1.        ]
position from extrinsics matrix:  [-5.10290077e+02 -2.02182912e+03  1.00000000e+00]
position from inverse matrix   :  [572.87374321 743.87228508   1.        ]
triangulated zero point:  [  302.94894188   385.77538056 -1800.92635594]


In [33]:
A = triangulator.get_DLT_matrix(point)
print('A: \n', A)
U, S, Vt = np.linalg.svd(A.T @ A, full_matrices=False)
print('s: \n', S)
print('Vt: \n', Vt)
vn = Vt[::, 3]
print('vn: \n', vn.reshape((4, 1)))
print('vn normalized: \n', (vn / vn[-1]).reshape((4, 1)))
print('all close: ', np.allclose(U @ np.diag(S) @ Vt, A))
print('w2: ', vn.T @ A.T @ A @ vn)

# reproject
reproj = projection @ vn / vn[-1]
reproj = reproj / reproj[-1]
reproj = reproj / [-1]
print('triangulated back: \n', reproj)

A: 
 [[ 8.00425265e+05 -8.79170561e+05  1.67557110e+05  3.98432446e+08]
 [ 1.21565001e+03 -6.43297025e+02  7.89048866e+01 -8.60414644e+03]
 [ 7.10855749e+05  1.02848966e+06  5.94792201e+05  4.59057911e+08]
 [ 3.92240566e+02  1.28408596e+03  3.00448919e+02 -5.51459163e+03]]
s: 
 [3.69484059e+17 1.87908948e+12 1.68062155e+06 1.58020589e+03]
Vt: 
 [[-1.74632857e-03 -3.29776265e-04 -9.19674450e-04 -9.99997998e-01]
 [ 1.01068158e-01 -9.76157051e-01 -1.92100328e-01  3.22085622e-04]
 [-9.81549277e-01 -6.63363278e-02 -1.79323439e-01  1.90090668e-03]
 [-1.62305869e-01 -2.06680418e-01  9.64852176e-01 -5.35753293e-04]]
vn: 
 [[-9.99997998e-01]
 [ 3.22085622e-04]
 [ 1.90090668e-03]
 [-5.35753293e-04]]
vn normalized: 
 [[ 1.86652702e+03]
 [-6.01182720e-01]
 [-3.54810079e+00]
 [ 1.00000000e+00]]
all close:  False
w2:  1940556512435.2886
triangulated back: 
 [-4.95179108e+03 -1.26951894e+03 -1.00000000e+00]


In [20]:
p1, p2, p3 = tuple(triangulator.projection_matrices[0])
# print(p1)
# print(p2)
# print(p3)
print(triangulator.projection_matrices[0])

[[ 1.08309071e+03 -1.18985701e+03  2.28652066e+02  5.40099812e+05]
 [-2.12327677e+01 -1.33766810e+02  1.41676714e+03  7.01315580e+05]
 [-2.27765129e-01 -9.39106497e-01  2.57297559e-01  9.42790307e+02]]


In [ ]:
caps = [
    create_capture_from_json(4, '../config/capture_params.json'),
    create_capture_from_json(2, '../config/capture_params.json')
]

In [1]:
import cv2


while True:
    idx = 0
    for cap in caps:
        ret, frame = cap.read()

        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners, ids, _ = cv2.aruco.detectMarkers(
            image=gray,
            dictionary=cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
        )
        if ids is not None and 0 in ids:
            for corner_idx in range(len(ids)):
                if ids[corner_idx] == 0:
                    cv2.aruco.drawDetectedMarkers(
                        frame, 
                        corners=corners, 
                        ids=ids
                        )
                    break
        idx += 1
        cv2.imshow(f'cam{idx}', cv2.resize(src=frame, dsize=None, fx=0.35, fy=0.35))
    if cv2.waitKey(10) == 27:
        break


cv2.imshow('huh', frame)
cv2.waitKey(1000)
cv2.destroyAllWindows()

NameError: name 'caps' is not defined

In [48]:
for cap in caps:
    cap.release()

cv2.destroyAllWindows()